In [28]:
import ds_loader as dsl
import numpy
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.models.mobilenet import mobilenet_v2
from pathlib import Path


In [29]:
class Config():
    batch_size = 32
    epochs = 20
    learn_rate = 0.001
    CLASSES = 55
    DATABASE_FOLDER_TRAIN = str(Path("../../dataset/training/"))
    DATABASE_FOLDER_TEST = str(Path("../../dataset/testing/"))
    SAMPLING_RATE = 800
    #weight_decay = 0.001

In [30]:
dsl.get_dataset(save=True, folder=Config.DATABASE_FOLDER_TRAIN)
dsl.get_dataset(save=True, folder=Config.DATABASE_FOLDER_TEST)

array([[[[-3.23840380e-01, -2.50998068e+00, -1.54999995e+00, ...,
          -5.67999983e+00, -1.11000001e+00, -1.02500000e+02],
         [-1.18244357e-01,  9.19208527e+00, -5.29999971e-01, ...,
           1.13299999e+01,  9.73999977e+00, -1.12804878e+00],
         [ 3.51537280e-02,  4.19128448e-01,  6.59999967e-01, ...,
           2.41720001e+02, -2.39999995e-01, -4.58841476e+01]],

        [[-3.30231994e-01, -2.37585950e+00, -1.33999991e+00, ...,
          -5.66575193e+00, -1.14999998e+00, -9.63932266e+01],
         [-1.52332813e-01,  9.22322083e+00, -5.29999971e-01, ...,
           1.13183870e+01,  9.73999977e+00, -1.61434448e+00],
         [ 1.17179090e-02,  5.02954125e-01,  6.10000014e-01, ...,
           2.41725311e+02, -2.09999993e-01, -4.63027534e+01]],

        [[-3.39819372e-01, -2.12917256e+00, -1.18999994e+00, ...,
          -5.62617159e+00, -1.17999995e+00, -8.30261536e+01],
         [-1.71507582e-01,  9.42679691e+00, -5.19999981e-01, ...,
           1.12855549e+01,  9.7299

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [24]:
train_dl = dsl.get_dataloader(is_train=True, batch_size=Config.batch_size)
test_dl = dsl.get_dataloader(is_train=False, batch_size=Config.batch_size)

In [ ]:
#define mode
model = mobilenet_v2(pretrained=True)

model.features[0][0] = nn.Conv2d(9, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=Config.CLASSES)

model.to(device)

# define optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr=Config.learn_rate, weight_decay=Config.weight_decay)
optimizer = torch.optim.Adam(model.parameters(), lr=Config.learn_rate)
# define loss function
loss_function = nn.CrossEntropyLoss()

next(model.parameters()).is_cuda

In [12]:
loss_ls_train = []
model.train()

for epoch in range(Config.epochs):
    for data, labels in train_dl:
        data, labels = data.to(device), labels.to(device)

        optimizer.zero_grad()
        data = data.permute(0, 3, 1, 2)

        prediction = model(data)

        loss = loss_function(prediction, labels)
        loss.backward()
        optimizer.step()

        loss_ls_train.append(loss.data.item())

    print('===> Epoch: {} loss: {:.4f}'.format(epoch, loss.data.item()))

plt.plot(loss_ls_train)
plt.show()

NameError: name 'model' is not defined

In [25]:
model = torch.load('/Users/falcolentzsch/Develope/MDS4AGT/krasses_model.pt',map_location=device)
# Start Evaluation
model.eval()
correct_pred = 0
num_pred = 0

for data, label in test_dl:
    data, label = data.to(device), label.to(device)

    data = data.permute(0, 3, 1, 2)
    pred = torch.argmax(model(data), dim=1)

    num_pred += len(label)
    correct_pred = correct_pred + torch.eq(pred,label).sum().data.item()

print('Number of correct predictions: ' + str(correct_pred))
print('Number of Predictions:' + str(num_pred))
print('Portion: ' + str(correct_pred/num_pred))

Number of correct predictions: 1284
Number of Predictions:2288
Portion: 0.5611888111888111


In [31]:
model.eval()


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(9, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen